In [1]:
import logging, os
logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

import tensorflow as tf

from pathlib import Path

import keras_tuner as kt

from importlib import reload
import hyper_model_creator
import keras_model_creator
import sentinel_utils

In [2]:
trials_dir = Path('trials', 'hyperband_resnet')
trial_metric = 'val_weightedf2score'

In [3]:
callbacks = [
    tf.keras.callbacks.TensorBoard(trials_dir.joinpath('board')),
    tf.keras.callbacks.EarlyStopping(
        monitor=trial_metric, patience=10, mode='max'
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor=trial_metric, factor=0.5, patience=4, min_lr=1e-6,
    ),
]

In [ ]:
reload(hyper_model_creator)

hypermodel = hyper_model_creator.BuildHyperModel()

tuner = kt.Hyperband(
    hypermodel,
    objective=kt.Objective(trial_metric, direction='max'),
    directory=trials_dir.parent,
    project_name=trials_dir.name,
    hyperband_iterations=3,
    max_epochs=100,
    overwrite=False,
)

tuner.search(
    verbose=1, 
    callbacks=callbacks
)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
False             |False             |class_weight
2017              |2017              |training_years
l2                |l2                |kernel_regularizer
0.1               |0.1               |spatial_dropout
relu              |relu              |activation
4                 |4                 |pool_size
0.3               |0.3               |dropout
True              |True              |bias_initializer
0.0001            |0.0001            |learning_rate
binary_crossent...|binary_crossent...|loss_function
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
4                 |4                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2


I0000 00:00:1723474789.927792   18506 service.cc:145] XLA service 0x7fc0b8003e10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1723474789.927829   18506 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Ti, Compute Capability 8.9
I0000 00:00:1723474800.328653   18506 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


3717/7100 ━━━━━━━━━━━━━━━━━━━━ 1:51 33ms/step - accuracy: 0.3213 - auc: 0.7236 - loss: 15.4892 - prc: 0.4559 - precision: 0.4992 - recall: 0.3974 - weightedf1score: 0.4383 - weightedf2score: 0.4121

In [ ]:
# tuner.results_summary(num_trials=2)

In [ ]:
# %load_ext tensorboard
# tb_path = str(trials_dir.joinpath('board'))
# %tensorboard --logdir "$tb_path"